In [214]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import datetime
from fake_useragent import UserAgent
import time
import json

from itertools import combinations
from difflib import SequenceMatcher

from google.cloud import bigquery
from google.cloud.bigquery import job
from tqdm import tqdm
import pandas as pd
import google.auth

from openai import OpenAI
import openai

import konlpy
from konlpy.tag import Kkma, Komoran, Okt, Hannanum

PROJCECT = 'ballosodeuk'
credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/bigquery",
    ]
)
bq = bigquery.Client(credentials=credentials,project=PROJCECT)

key = pd.read_csv('./aikey.csv')['key'].values # OpenAI Secret
okt = Okt()


실험 검색어 테이블

In [7]:
search_df

,sub_id,product_name,ori_url,product_id,item_id,vendoritemid
0,270f7075-5d2f-3aaf-bdc3-5e2e240608e4,대도갤러리 키높이 올루바 긴문형 12자 장롱세트,https://www.coupang.com/vp/products/100064921?...,100064921,305756598,70316548415
1,6213b1e4-21e3-34ec-bbd3-1f009478fc59,1+1 특가이벤트 춘하추동 흑풍 올림찌,https://www.coupang.com/vp/products/100076920?...,100076920,305787463,3754104624
2,69d4af9d-470a-3de0-8b60-737079cc5407,"큐빅스 큐빅스2 공간박스 B2147, 화이트, 1개",https://www.coupang.com/vp/products/100077087?...,100077087,0,3754105606
3,11a40d76-62fc-3d86-93a7-ab46bca25475,"베트랑 대형견용 소가죽 개껌, 우유맛, 1개",https://www.coupang.com/vp/products/1000867?it...,1000867,23194743114,NaN
4,5695cd53-1315-302e-be61-56272de0b581,피부과전용 사해소금 진정 하이드로컴플렉스 수딩 아쿠아겔 얼굴 열감 레이저시술후 홍조,https://www.coupang.com/vp/products/100089345?...,100089345,21472322165,86078343492
...,...,...,...,...,...,...
49995,505811aa-c6f4-3306-aa43-073c1f9464eb,1+1이벤트&단품 매트리스 방수 커버 / 침대 커버 / 방수 매트리스 커버 / 진드...,https://www.coupang.com/vp/products/5890998843...,5890998843,10359064807,72441237950
49996,33c255d1-6134-34ce-a257-6c9d28f38268,1+1이벤트&단품 매트리스 방수 커버 / 침대 커버 / 방수 매트리스 커버 / 진드...,https://www.coupang.com/vp/products/5890998843...,5890998843,10359064869,72441237917
49997,36e489d6-daff-3a2c-a11d-3f25f9fc253b,"힘찬쇼핑_ 칼라 둥근 굵은 고무줄 고무끈 3m 7색 쫀쫀하고 질긴 고무줄, 검정, 1개",https://www.coupang.com/vp/products/5891062212...,5891062212,10359371181,77641525479
49998,3567c8e8-ae5f-34fd-b14d-81b4e7b8de8a,"숨쉬는 연잎밥 150g x 42개 간편 영양 낱개포장, 150g, 42개",https://www.coupang.com/vp/products/5891079868...,5891079868,6889575025,74182120424


쿠팡 크롤링 코드

In [30]:
def get_data(base_url, retries=5):
    with requests.Session() as session:
        session.headers.update(set_header())  # 세션에 공통 헤더 설정
        for i in range(retries):
            response = session.get(base_url)
            print(f"Attempt {i + 1}: Status code {response.status_code}")
            
            if response.status_code == 200:
                soup = parse_data(response.text)
                
                try:
                    # 'ul.search-product-list'가 실제로 존재하는지 확인
                    search_product_list = soup.select('ul.search-product-list')
                    if not search_product_list:
                        raise ValueError("search-product-list not found in the page")

                    data_products = json.loads(search_product_list[0].get('data-products'))
                    if len(data_products['indexes']) > 0:
                        print(len(data_products['indexes']))
                        print(data_products['indexes'])
                        print("Good to go")
                        return soup
                    else:
                        print(f"Attempt {i + 1}: Required data not found, retrying...")
                except Exception as e:
                    print(f"Attempt {i + 1}: Error while processing data: {e}, retrying...")
            else:
                print(f"Attempt {i + 1}: Failed with status code {response.status_code}, retrying...")

            randint = lagging_time()
            print(f"Retrying in {randint} seconds...")

    print("Failed to retrieve fully loaded page after multiple attempts")
    return None


def parse_data(data):
    soup = BeautifulSoup(data, 'html.parser')
    return soup

# def set_header():
#     return {
#         "User-Agent" : UserAgent().random,
#         "Accept-Language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
#     }

def set_header():
    return {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding' : 'gzip, deflate, br, zstd',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }

def lagging_time() -> None:
    rand = random.uniform(3,5)
    # randint = random.randint(3,8)
    time.sleep(rand)
    return rand

def get_parse_data(data):
    page_item_list = data.select('ul.search-product-list')
    page = data.select('ul.search-product-list')
    name = data.select('ul.search-product-list>li>a>dl>dd>div>div.name')
    ids = data.select('ul.search-product-list>li')
    a = data.select('ul.search-product-list>li>a')

    for i in range(len(page_item_list)):
        query_quality = page[i].get('data-srp-ranking-badge')
        query = page[i].get('data-query')

        data_product_id = ids[i].get('data-product-id')
        data_item_id = a[i].get('data-item-id')
        data_is_soldout = ids[i].get('data-is-soldout')
        data_vendor_item_id = ids[i].get('data-vendor-item-id')
        data_winner_vendor_item_id = ids[i].get('data-winner-vendor-item-id')
        prd_name = name[i].text.strip()

        print(data_product_id,data_item_id,data_vendor_item_id)

    return [query,prd_name,query_quality,data_is_soldout,data_product_id,data_vendor_item_id,data_winner_vendor_item_id]


query = "컴프로 남성 스포츠 중목 쿠션 이중바닥 링글 기능성 양말 5족"
base_url = f"https://www.coupang.com/np/search?component=&q={query}"
data = get_data(base_url,5)
outs = get_parse_data(data)
outs


Attempt 1: Status code 200
49
[1083222205, 1083222205, 1083222205, 1083222205, 7225618263, 7170749211, 1083222205, 1083222205, 7225618263, 1083222205, 1083222205, 7227281069, 7225618263, 7967167220, 7260515029, 6638406272, 7260513478, 7225618263, 6431129537, 8020469061, 7170749211, 7916879592, 8094084102, 7212514545, 6638406272, 8065731818, 7225618263, 7225618263, 7260513478, 7260513478, 7954029719, 1085883139, 7260513478, 7260513478, 7225618263, 7260513478, 8205357916, 7225618263, 8020469100, 7883588292, 1085883139, 7883588292, 7883588292, 6431129537, 7107178669, 6643644946, 7310230793, 8273425575, 8065731818]
Good to go
1083222205 2036520922 89625816099


['컴프로 남성 스포츠 중목 쿠션 이중바닥 링글 기능성 양말 5족',
 '컴프로 남성 스포츠 중목 쿠션 이중바닥 링글 기능성 양말 5족, 흰색-흰색',
 'A',
 '',
 '1083222205',
 '89625816099',
 '89625816099']

실험 검색어 및 쿼리 분해조합

In [3]:
search_df = pd.read_csv(r"C:\Users\Owner\Downloads\240902_cp_sampledata.csv",dtype=str)
search_df.head(2)

,sub_id,product_name,ori_url,product_id,item_id,vendoritemid
0,270f7075-5d2f-3aaf-bdc3-5e2e240608e4,대도갤러리 키높이 올루바 긴문형 12자 장롱세트,https://www.coupang.com/vp/products/100064921?...,100064921,305756598,70316548415
1,6213b1e4-21e3-34ec-bbd3-1f009478fc59,1+1 특가이벤트 춘하추동 흑풍 올림찌,https://www.coupang.com/vp/products/100076920?...,100076920,305787463,3754104624


In [8]:
def get_randQuqery():
    rand = random.randint(1,len(search_df))
    rand_query = list(search_df['product_name'])[rand]
    return rand_query

def set_query():
    split_query = re.split(",",get_randQuqery())
    
    ## 형태소 분리
    print(split_query)
    forward =  split_query[0][:10]
    backward = split_query[0][-10:]
    print(okt.morphs(split_query[0]))
    print(forward)
    print(backward)
# get_randQuqery()
set_query()

# 검증
# print(len(rand_query.replace(" ","")))




['건건당 야들야들 오징어실채300g']
['건건', '당', '야들야들', '오징어', '실채', '300', 'g']
건건당 야들야들 오
 오징어실채300g


In [118]:
tst = get_randQuqery()
print(tst)
okt.phrases(tst)


로더베르 시그니처 퍼퓸 샴푸 베이비파우더 1000g, 1000ml, 4개


['로더베르',
 '로더베르 시그니처',
 '로더베르 시그니처 퍼퓸',
 '로더베르 시그니처 퍼퓸 샴푸',
 '로더베르 시그니처 퍼퓸 샴푸 베이비파우더',
 '로더베르 시그니처 퍼퓸 샴푸 베이비파우더 1000g',
 '1000ml',
 '4개',
 '로더',
 '베르',
 '시그니처',
 '퍼퓸',
 '샴푸',
 '베이비',
 '파우더',
 '1000g']

In [199]:
original_sentence = get_randQuqery()
phrases = okt.phrases(original_sentence)

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def find_top_combinations(original, phrases, max_len=11, top_n=5):
    combinations_list = []
    
    for r in range(1, len(phrases) + 1):
        for combo in combinations(phrases, r):
            combined_phrase = " ".join(combo).replace(" ", "")
            if len(combined_phrase) <= max_len:
                sim = similarity(original, combined_phrase)
                combinations_list.append((" ".join(combo), sim))
    
    combinations_list.sort(key=lambda x: x[1], reverse=True)
    return combinations_list[:top_n]

top_combinations = find_top_combinations(original_sentence, phrases)
top_combinations
# for idx, (sentence, sim) in enumerate(top_combinations, 1):
#     print(f"Rank {idx}: {sentence} (Similarity: {sim:.4f})")


['쥬골드', '쥬골드 14K', '쥬골드 14K 컷팅', '쥬골드 14K 컷팅 물방울2', '쥬골드 14K 컷팅 물방울2 원터치', '쥬골드 14K 컷팅 물방울2 원터치 링귀걸이', '쥬골드 14K 컷팅 물방울2 원터치 링귀걸이 14K귀걸이', '골드', '14K', '컷팅', '물방울', '원터치', '귀걸이'] 쥬골드 14K 컷팅 물방울2 원터치 링귀걸이 14K귀걸이


[('쥬골드 14K 컷팅 14K', 0.5238095238095238),
 ('쥬골드 14K 컷팅 물방울', 0.5238095238095238),
 ('쥬골드 14K 컷팅 원터치', 0.5238095238095238),
 ('쥬골드 14K 컷팅 귀걸이', 0.5238095238095238),
 ('쥬골드 14K 컷팅 물방울', 0.5238095238095238)]

Search API 코드

In [15]:
info = pd.read_csv('./adm_data.csv')
adm_url = info['url'].values[0]
adm_id = info['id'].values[0]
adm_pw = info['pw'].values[0]

info = pd.read_csv('./dw_data.csv')
dw_url = info['url'].values[0]
dw_id = info['id'].values[0]
dw_pw = info['pw'].values[0]
dw_tag = info['tag'].values[0]
dw_apilink = info['apilink'].values[0]


'https://adapi.inlcorp.com/cou/api_reco.php?code=balsodw1&adid='

In [137]:
def get_json_from_url(query) -> json : 
    url = f'https://api.adjoin.co.kr/cou/api.php?code=balsosa1&q={query}&c=10'
    try:
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json() 
        return data
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}") 
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

def get_parse_itemId(items,order) -> list :
    if order == 'itemid':
        parsed = re.search(r'itemId%3D(\d+)', items).group(1)
    elif order == 'verdorItemId':
        parsed = re.search(r'vendorItemId%3D(\d+)', items).group(1)
    
    return parsed

def set_table(json_data) -> pd.DataFrame :
    product_name = [str(item['productName']) for item in json_data['data']['productData']]
    product_id = [str(item['productId']) for item in json_data['data']['productData']]
    item_id = [get_parse_itemId(item['productUrl'],'itemid') for item in json_data['data']['productData']]
    vendor_id = [get_parse_itemId(item['productUrl'],'verdorItemId') for item in json_data['data']['productData']]

    api_df = pd.DataFrame({
        'product_name' : product_name,
        'product_id' : product_id,
        'item_id' : item_id,
        'vendor_id' : vendor_id})
    
    return api_df

# tst = get_randQuqery()
# query = tst[:10]
query = '오토바이 배달 조끼 낚시'
# print(tst)
# print(query)
api_df = set_table(get_json_from_url(query))

api_df

,product_name,product_id,item_id,vendor_id
0,잇다 1+1+1 여름 자외선차단 자전거 오토바이 바라클라바,8107022643,22950135994,89984286074
1,머지샵 남성용 오토바이 우비 상하의 세트,7935011671,21840326205,88888663930
2,더빅보스 봄가을 남성용 WD20멀티포켓조끼 작업복 등산조끼,7090124303,17665557489,71283605753
3,[양손을 자유롭게] 시원한 배달대행 반사 조끼,8004874462,22309077568,89354503586
4,[더빅보스] 넉넉한 수납 다용도 조끼 (1+1) _ 남성용 작업복 다용도 베스트 빅...,7634881952,20271654341,83634200079
5,[더빅보스] NB기모조끼 (1+1) _겨울 남성 작업복 등산 조끼 따뜻한 기모안감 ...,6096404041,11395454530,78671506102
6,제트지엠 오토바이 배달 자석 조끼,7712995454,20680085227,87969478054
7,노블진 주머니많은 현장 작업복조끼 노가다,6273364705,12841989770,80107555915
8,KLIFE 선풍기 조끼 에어컨 아이스 냉각옷 쿨 냉풍조끼 여름작업복 얼음조끼 남녀공...,8213535829,23584617628,90610504706
9,ZB-V2422 지벤 여름용 선풍기 조끼 풀세트,7935140094,21840870071,89929449481


In [72]:
tst

'좋은느낌 수퍼롱 오리지널 생리대 날개형, 오버나이트이상, 20개입, 2개'

gpt api

In [193]:
client = OpenAI(api_key=key[1], organization='org-b1AmHOsZydHwAp7obHQA860F')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [77]:
get_randQuqery()

'월드가드닝 식물뿌리용 보약 영양제, 300ml, 5개'

검색 로직

In [215]:
class coupang_sa():
    def __init__(self,dataset):
        self.search_df = dataset
        self.random_queryset = self.get_randQuery()
        self.queryset = self.set_query()
        self.querymorphs = self.get_morps()
    
    ## 모수 추출
    def get_randQuery(self) -> list : #
        rand = random.randint(1,len(search_df))
        rand_query = list(search_df['product_name'])[rand]
        return rand_query

    ## 키워드 추출 basic
    def set_query(self):
        split_query = re.split(",",self.random_queryset)
        
        ## 형태소 분리
        # print(split_query)
        forward =  split_query[0][:10]
        backward = split_query[0][-10:]
        # print(okt.morphs(split_query[0]))
        # print(forward)
        # print(backward)

        return split_query
    
    def get_morps(self):
        phrases = okt.phrases(self.random_queryset)
        return phrases

    def test(self):
        return self.queryset, self.querymorphs

    ## 키워드 추출 intermediate
    def similarity(self, a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    def find_top_combinations(self, max_len=11, top_n=5):
        combinations_list = []
        # print(self.querymorphs)
        # print(self.random_queryset)
        for r in range(1, len(self.querymorphs) + 1):
            for combo in combinations(self.querymorphs, r):
                combined_phrase = " ".join(combo).replace(" ", "")
                if len(combined_phrase) <= max_len:
                    sim = similarity(self.random_queryset, combined_phrase)
                    combinations_list.append((" ".join(combo), sim))

        print(111222,self.querymorphs, self.random_queryset)
        combinations_list.sort(key=lambda x: x[1], reverse=True)
        return combinations_list[:top_n]
    
    # 서치api 작동
    def get_json_from_url(self, query) -> json : 
        url = f'https://api.adjoin.co.kr/cou/api.php?code=balsosa1&q={query}&c=10'
        try:
            response = requests.get(url)
            response.raise_for_status()  
            data = response.json() 
            return data
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}") 
        except Exception as err:
            print(f"An error occurred: {err}")
        return None
    
    def get_parse_itemId(self, items,order) -> list :
        if order == 'itemid':
            parsed = re.search(r'itemId%3D(\d+)', items).group(1)
        elif order == 'verdorItemId':
            parsed = re.search(r'vendorItemId%3D(\d+)', items).group(1)
        
        return parsed

    def set_table(self, json_data) -> pd.DataFrame :
        product_name = [str(item['productName']) for item in json_data['data']['productData']]
        product_id = [str(item['productId']) for item in json_data['data']['productData']]
        item_id = [get_parse_itemId(item['productUrl'],'itemid') for item in json_data['data']['productData']]
        vendor_id = [get_parse_itemId(item['productUrl'],'verdorItemId') for item in json_data['data']['productData']]

        api_df = pd.DataFrame({
            'product_name' : product_name,
            'product_id' : product_id,
            'item_id' : item_id,
            'vendor_id' : vendor_id})
        
        return api_df
    


In [226]:
dataset = pd.read_csv(r"C:\Users\Owner\Downloads\240902_cp_sampledata.csv",dtype=str)

cp = coupang_sa(dataset)
# cp.test()
cp.find_top_combinations()


111222 ['양념', '양념 무뼈', '양념 무뼈 닭발', '양념 무뼈 닭발 350g', '350g', '10개', '무뼈', '닭발', '10'] 냠냠 양념 무뼈 닭발 350g, 350g, 10개


[('양념 무뼈 350g 10개', 0.5789473684210527),
 ('양념 무뼈 닭발 350g', 0.5405405405405406),
 ('양념 무뼈 닭발 350g', 0.5405405405405406),
 ('양념 무뼈 350g 10', 0.5405405405405406),
 ('양념 무뼈 닭발 10개', 0.5)]